In [16]:
import os

import dotenv
import pydantic
from langchain.agents import AgentExecutor, initialize_agent, AgentType
from langchain.agents.tools import Tool
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI

# assuming .env in root of repo contains OPENAI_API_KEY
dotenv.load_dotenv("../../../../../.env")

llm = ChatOpenAI(temperature=0)
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

Define tools which provide:
- The `n`th prime number (using a small subset for this example) 
- The LLMMathChain to act as a calculator

In [14]:
primes = {
    998:7901,
    999:7907,
    1000:7919
}

def get_prime(n: int, primes: dict = primes) -> int:
    return primes.get(n)

async def aget_prime(n: int, primes: dict = primes) -> int:
    return primes.get(n)

class CalculatorInput(pydantic.BaseModel):
    question: str = pydantic.Field()

tools = [
    Tool(
        name="GetPrime",
        func=get_prime, 
        description="A tool that returns the `n`th prime number",
        coroutine=aget_prime,
    ),
    Tool.from_function(
        func=llm_math_chain.run,
        name="Calculator",
        description="useful for when you need to answer questions about math",
        args_schema=CalculatorInput,
        coroutine=llm_math_chain.arun
    )
]

In [17]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [47]:
question = (
    "What are the 998th, 999th and 1000th prime numbers?"
    #"What is the product of the 998th, 999th and 1000th prime numbers?"
)

In [30]:
ans = agent.run(question)

In [48]:
for ix, step_output in enumerate(agent.iter(question)):
    print(f"STEP {ix}:")
    print(step_output)
    

STEP 0:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='I can use the GetPrime tool to find the prime numbers.\nAction: GetPrime\nAction Input: 998'), None)]}


OutputParserException: Could not parse LLM output: `I need to find the 998th prime number first.`

In [46]:
for ix, step_output in enumerate(agent.iter(question)):
    print(f"STEP {ix}:")
    print(step_output)
    

STEP 0:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.\nAction: GetPrime\nAction Input: 998'), None)]}
STEP 1:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.\nAction: GetPrime\nAction Input: 998'), None), (AgentAction(tool='GetPrime', tool_input='998', log='I need to find the 998th prime number.\nAction: GetPrime\nAction Input: 998'), None)]}
STEP 2:
{'intermediate_steps': [(AgentAction(tool='GetPrime', tool_input='998', log='To find the product of the 998th, 999th, and 1000th prime numbers, I can use the GetPrime tool to find each prime number and then multiply them together.\nAction: GetPrime\nAction Input: 998'), None), (

In [21]:
from functools import reduce

In [22]:
reduce(lambda x, y: x*y, primes.values())

494725326233

In [36]:
7901*7907*7919

494725326233

In [38]:
494725326233 / 1,492,313,491

(494725326233.0, 492, 313, 491)

In [40]:
494725326233 / 1492313491

331.5156830093952